In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240617/sc-E2G/test/results/PBMC9/12_CD14.Mono.2.multiome_7features/Kendall/Pairs.tsv.gz"
path.pairs.ABC = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240617/sc-E2G/test/results/PBMC9/12_CD14.Mono.2.multiome_7features/Predictions/EnhancerPredictionsAllPutative.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240617/sc-E2G/test/results/PBMC9/12_CD14.Mono.2.multiome_7features/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/3.annotate_cell_type/RNA_count_matrix/12_CD14.Mono.2.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/2.Genome_wide_prediction/SCENT/PBMC9.12_CD14.Mono.2.100.240702/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ABC results

In [5]:
pairs.ABC = readGeneric(path.pairs.ABC,
                        header = T,
                        keep.all.metadata = T)

Filter ABC results which distance < 1M

In [6]:
pairs.ABC.1M = pairs.ABC[pairs.ABC$distance < 10^6]

Filter E-G pairs overlaping with pairs.ABC.1M

In [7]:
df.pairs.E2G.chr_rename = as.data.frame(pairs.E2G)[,1:3]
df.pairs.E2G.chr_rename[,"seqnames"] = paste(seqnames(pairs.E2G),
                                             mcols(pairs.E2G)[,"TargetGene"],
                                             sep = "_")
pairs.E2G.chr_rename = GRanges(df.pairs.E2G.chr_rename)
rm(df.pairs.E2G.chr_rename)

df.pairs.ABC.1M.chr_rename = as.data.frame(pairs.ABC.1M)[,1:3]
df.pairs.ABC.1M.chr_rename[,"seqnames"] = paste(seqnames(pairs.ABC.1M),
                                                mcols(pairs.ABC.1M)[,"TargetGene"],
                                                sep = "_")
pairs.ABC.1M.chr_rename = GRanges(df.pairs.ABC.1M.chr_rename)
rm(df.pairs.ABC.1M.chr_rename)

pairs.E2G.filter = pairs.E2G[countOverlaps(pairs.E2G.chr_rename,
                                           pairs.ABC.1M.chr_rename) > 0]
rm(pairs.E2G.chr_rename)
rm(pairs.ABC.1M.chr_rename)
pairs.E2G.filter

GRanges object with 2906945 ranges and 3 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100028774-100029108      * |         AGL
        [2]     chr1 100028774-100029108      * |      CDC14A
        [3]     chr1 100028774-100029108      * |         DBT
        [4]     chr1 100028774-100029108      * |        DPH5
        [5]     chr1 100028774-100029108      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2906941]     chrX     9986627-9987289      * |       TBL1X
  [2906942]     chrX     9986627-9987289      * |        WWC3
  [2906943]     chrX     9995789-9996213      * |       CLCN4
  [2906944]     chrX     9995789-9996213      * |       TBL1X
  [2906945]     chrX     9995789-9996213      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10

Import ATAC matrix

In [8]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [9]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

In [10]:
pairs.E2G.filter2 = pairs.E2G.filter[pairs.E2G.filter$TargetGene %in% rownames(matrix.rna) &
                                     pairs.E2G.filter$PeakName %in% rownames(matrix.atac)]

Prepare SCENT meta data

In [11]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = "K562")
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
Multiome10x_05_AAACAGCCAACAACAA-1,Multiome10x_05_AAACAGCCAACAACAA-1,2924,530,K562,7.980708,0.18125855
Multiome10x_05_AAACAGCCATCCTAGA-1,Multiome10x_05_AAACAGCCATCCTAGA-1,4553,608,K562,8.423542,0.13353833
Multiome10x_05_AAACATGCAAAGGTAC-1,Multiome10x_05_AAACATGCAAAGGTAC-1,3957,547,K562,8.283241,0.13823604
Multiome10x_05_AAACATGCAGGACCAA-1,Multiome10x_05_AAACATGCAGGACCAA-1,2856,404,K562,7.957177,0.14145658
Multiome10x_05_AAACCAACAACTAACT-1,Multiome10x_05_AAACCAACAACTAACT-1,4542,513,K562,8.421123,0.11294584
Multiome10x_05_AAACCAACATGTTTGG-1,Multiome10x_05_AAACCAACATGTTTGG-1,3298,447,K562,8.101072,0.13553669
Multiome10x_05_AAACGGATCGTTAACA-1,Multiome10x_05_AAACGGATCGTTAACA-1,3043,425,K562,8.020599,0.13966480
Multiome10x_05_AAACGTACACTAGGTC-1,Multiome10x_05_AAACGTACACTAGGTC-1,3781,569,K562,8.237744,0.15048929
Multiome10x_05_AAAGCACCAGCTTAAT-1,Multiome10x_05_AAAGCACCAGCTTAAT-1,4365,627,K562,8.381373,0.14364261


RUN SCENT prediction

In [12]:
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            bs = boot::boot(df2, assoc, R = 100, formula = formula, 
                stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        }
    }
    object@SCENT.result <- res
    return(object)
}

In [13]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [14]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter2)))
chr.run = chr.run[!chr.run %in% chr.done]

In [15]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter2[seqnames(pairs.E2G.filter2) == chr.tmp]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna[rownames(matrix.rna) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [16]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter2)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [17]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 496679 ranges and 11 metadata columns:
       seqnames              ranges strand |  TargetGene               PeakName
          <Rle>           <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 100037723-100039165      * |      CDC14A chr1-100037723-10003..
  chr1     chr1 100037723-100039165      * |        RTCA chr1-100037723-10003..
  chr1     chr1 100037723-100039165      * |     SLC30A7 chr1-100037723-10003..
  chr1     chr1 100037723-100039165      * |     SLC35A3 chr1-100037723-10003..
  chr1     chr1 100037723-100039165      * |      TRMT13 chr1-100037723-10003..
   ...      ...                 ...    ... .         ...                    ...
  chrX     chrX     9981040-9982043      * |        WWC3   chrX-9981040-9982043
  chrX     chrX     9986627-9987289      * |       TBL1X   chrX-9986627-9987289
  chrX     chrX     9986627-9987289      * |        WWC3   chrX-9986627-9987289
  chrX     chrX     9995789-9996213      * |       TBL1X   ch

In [18]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100037723,100039165,CDC14A,K562,0.130626923,0.06124682,2.13279523,0.03294153,0.08,1.09691001,1,1.09691001
chr1,100037723,100039165,RTCA,K562,0.039701785,0.13650727,0.29084008,0.77117363,0.58,0.23657201,1,0.23657201
chr1,100037723,100039165,SLC30A7,K562,0.112013477,0.07488933,1.49572008,0.13472662,0.18,0.74472749,1,0.74472749
chr1,100037723,100039165,SLC35A3,K562,0.172543466,0.14896626,1.15827213,0.24675300,0.22,0.65757732,1,0.65757732
chr1,100037723,100039165,TRMT13,K562,-0.204355281,0.15730888,-1.29907022,0.19391983,0.22,0.65757732,-1,-0.65757732
chr1,100046417,100047254,CDC14A,K562,0.082390262,0.08379488,0.98323746,0.32549058,0.32,0.49485002,1,0.49485002
chr1,100046417,100047254,RTCA,K562,-0.197819041,0.20579178,-0.96125821,0.33642235,0.22,0.65757732,-1,-0.65757732
chr1,100046417,100047254,SLC30A7,K562,0.013535172,0.10520788,0.12865170,0.89763326,0.92,0.03621217,1,0.03621217
chr1,100046417,100047254,SLC35A3,K562,0.332845573,0.18809775,1.76953509,0.07680462,0.14,0.85387196,1,0.85387196


In [19]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
[1] Matrix_1.6-5         GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
[4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGener